# Financial Analysis

---

## Setup

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_type = os.environ.get("OPENAI_API_TYPE")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get("OPENAI_API_VERSION")

## LLM

In [ ]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="gpt40125",
    temperature=0,
)

## Creating Tools

### crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
from crewai import Agent, Task, Crew

In [6]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
)

In [9]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    llm=llm,
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
)

## Creating Tasks

In [10]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [11]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [12]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [13]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew

- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [14]:
from crewai import Crew, Process


# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[
        data_analyst_agent,
        trading_strategy_agent,
        execution_agent,
        risk_management_agent,
    ],
    tasks=[
        data_analysis_task,
        strategy_development_task,
        execution_planning_task,
        risk_assessment_task,
    ],
    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True,
)

## Running the Crew

In [15]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [16]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
Thought: To effectively monitor and analyze market data for AAPL, I need to break down the task into specific components that can be handled by my team members based on their expertise. First, I need to gather and preprocess the market data. Then, I'll need to apply statistical modeling and machine learning to identify trends and predict market movements. Finally, I'll need to interpret these findings to identify market opportunities or threats.

Action: Delegate work to co-worker
Action Input: {"task": "Gather and preprocess market data for AAPL", "context": "We need to continuously monitor and analyze market data for AAPL. This involves collecting real-time and historical market data, cleaning it to remove any i

In [17]:
from IPython.display import Markdown

Markdown(result)

**AAPL Trading Strategy Risk Analysis Report**

**Executive Summary:**
This report provides a comprehensive risk analysis of the proposed trading strategies and execution plans for AAPL, focusing on day trading opportunities and medium risk tolerance. Based on the insights gathered from our Data Analyst, Trading Strategy Developer, and Risk Advisor, we have identified key areas of potential risk and proposed strategies to mitigate these risks effectively.

**1. Technical Analysis Risks:**
- **Unreliability of Technical Indicators:** The current bearish market trends and the breakdown of AAPL's stock below major uptrend lines since late 2018 indicate that traditional technical indicators may not be as reliable in predicting market movements. The observed bearish developments suggest a long-term negative trend, making it crucial to approach technical analysis with caution.
- **Mitigation Strategies:** Diversify the technical indicators used for analysis and incorporate alternative data sources, such as macroeconomic indicators and industry trends, to enhance decision-making. Continuous monitoring and adjustment of technical analysis strategies are recommended to adapt to changing market conditions.

**2. LSTM Network and Deep Learning Enhancements Risks:**
- **Predictive Unpredictability:** While LSTM network predictions, especially when integrated with sentiment analysis, show promise in forecasting AAPL's short-term price movements, the inherent unpredictability of the stock market poses a significant risk. Unexpected events or changes in investor behavior can lead to discrepancies between predicted and actual stock movements.
- **Mitigation Strategies:** Use LSTM network predictions as part of a broader investment strategy that considers various data sources and analytical methods. Implement a rigorous backtesting regime to validate the effectiveness of LSTM models continuously and adjust parameters as necessary to improve accuracy.

**3. Risk Management Strategies:**
- **Adequacy of Proposed Measures:** The proposed risk management strategies, including setting a stop-loss level at approximately 1% below the entry point, implementing a risk-reward ratio of at least 1:2, and limiting the total capital allocated to AAPL trades to no more than 10% of the day trading portfolio, are deemed adequate to mitigate potential losses in the current market environment.
- **Recommendations:** It is crucial to remain vigilant and flexible, ready to adjust risk management strategies as market conditions evolve. Continuous monitoring of market trends and the performance of trading strategies will be essential in ensuring that risk management measures remain effective and aligned with the firm's risk tolerance.

**Conclusion:**
The proposed trading strategies and execution plans for AAPL present several risks, primarily due to the current bearish market trends and the unpredictability of predictive models. However, by implementing the suggested mitigation strategies and maintaining a flexible approach to risk management, we can navigate these challenges effectively. Continuous learning and adaptation to market conditions will be key to the success of our trading strategies for AAPL.

This report underscores the importance of a cautious and informed approach to trading AAPL, emphasizing the need for continuous evaluation and adjustment of strategies in response to market developments.